In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
tqdm.pandas()

In [19]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [20]:
print("Shape of Train: ", train.shape)
train.head()

Shape of Train:  (450000, 20)


,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [21]:
train['cancelled'].value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

Feature Ideas:
- Time to Allot Order to Rider
- Time taken by rider to accept order
- Time for Food Preparation
- Time taken for delivery from time of order
- Check if there is an overlap between rider ID in train and test, if there is, add a column for rider cancellation rate
- If clustering happens for time of day of cancellation, add as a feature
- Ratio of first mile to last mile distance

Cleaning Ideas:
- lifetime_order_count change as per date
- interpolate based on rider_id for session time

Future Ideas:
- Subsample first and then smote in order to prevent smote from messing up

In [22]:
train[train['accept_time'].isna()].head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
1700,2021-01-26 06:35:16,558453,2021-01-26 00:00:00,2021-01-26 06:35:18,NaN,NaN,NaN,15903,1.8026,8.55,50.0,50.0,1,0.0,1216.0,NaN,NaN,NaN,51.300000,2021-01-26 06:40:17
4066,2021-01-26 07:20:44,560819,2021-01-26 00:00:00,2021-01-26 07:20:45,NaN,NaN,NaN,3430,1.6112,4.29,29.0,29.0,1,0.0,190.0,NaN,NaN,NaN,52.700000,2021-01-26 07:21:20
9296,2021-01-26 08:58:12,566050,2021-01-26 00:00:00,2021-01-26 08:58:41,NaN,NaN,NaN,20621,2.8689,4.07,NaN,NaN,1,NaN,917.0,NaN,NaN,NaN,120.733333,2021-01-26 09:00:35
13365,2021-01-26 12:42:36,570118,2021-01-26 00:00:00,2021-01-26 12:42:37,NaN,NaN,NaN,10605,0.7966,3.14,11.0,11.0,1,0.0,1546.0,NaN,NaN,NaN,478.333333,2021-01-26 12:47:37
13429,2021-01-26 12:45:42,570182,2021-01-26 00:00:00,2021-01-26 12:45:43,NaN,NaN,NaN,3585,0.9797,7.79,67.0,65.0,1,2.0,144.0,NaN,NaN,NaN,325.666667,2021-01-26 12:47:38


In [23]:
print("Shape of Test: ", test.shape)
test.head()

Shape of Test:  (144844, 16)


,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.6585,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.0709,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.3884,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.9039,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.8275,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333


In [24]:
len(set(train['rider_id']).intersection(set(test['rider_id'])))

11652

In [25]:
len(test['rider_id'].unique())

13682

In [26]:
train[train['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
315699,2021-02-03 06:43:10,239802,2021-02-03 00:00:00,2021-02-03 06:43:25,2021-02-03 06:43:30,2021-02-03 06:55:20,2021-02-03 07:12:18,1,1.4561,4.14,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,147.533333,NaN
328569,2021-02-03 13:38:38,252675,2021-02-03 00:00:00,2021-02-03 13:38:55,2021-02-03 13:40:17,2021-02-03 14:06:26,2021-02-03 14:46:21,1,2.4127,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,510.566667,NaN
331400,2021-02-03 14:02:48,255506,2021-02-03 00:00:00,2021-02-03 14:04:44,2021-02-03 14:05:50,2021-02-03 14:15:04,2021-02-03 14:33:03,1,0.8029,3.68,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,534.733333,NaN
346038,2021-02-03 16:14:20,270142,2021-02-03 00:00:00,2021-02-03 16:14:55,2021-02-03 16:15:50,2021-02-03 16:31:13,2021-02-03 16:40:45,1,1.1215,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,666.266667,NaN
348876,2021-02-03 16:41:11,272980,2021-02-03 00:00:00,2021-02-03 16:41:47,2021-02-03 16:42:43,2021-02-03 16:49:22,2021-02-03 16:58:00,1,0.4628,1.51,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,693.116667,NaN
352353,2021-02-03 17:14:48,276457,2021-02-03 00:00:00,2021-02-03 17:15:39,2021-02-03 17:16:26,2021-02-03 17:28:24,2021-02-03 17:35:16,1,1.1628,1.36,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,726.733333,NaN
353553,2021-02-03 17:26:31,277657,2021-02-03 00:00:00,2021-02-03 17:26:55,2021-02-03 17:27:51,2021-02-03 17:41:59,2021-02-03 18:04:50,1,1.0028,3.59,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,738.450000,NaN
373828,2021-02-04 14:14:41,216800,2021-02-04 00:00:00,2021-02-04 14:15:41,2021-02-04 14:16:31,2021-02-04 14:29:13,2021-02-04 14:41:52,1,0.8283,1.91,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,519.200000,NaN
379722,2021-02-04 15:10:50,222695,2021-02-04 00:00:00,2021-02-04 15:11:47,2021-02-04 15:12:24,2021-02-04 15:15:29,2021-02-04 15:23:42,1,0.2979,0.58,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,575.350000,NaN
383660,2021-02-04 15:48:53,226632,2021-02-04 00:00:00,2021-02-04 15:49:39,2021-02-04 15:49:52,2021-02-04 15:55:55,2021-02-04 16:04:07,1,0.8439,0.62,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,613.400000,NaN


In [27]:
test[test['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
5451,2021-02-06 13:42:02,135683,2021-02-06 00:00:00,2021-02-06 13:42:48,2021-02-06 13:44:05,1,0.669800,4.93,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,600.333333
17324,2021-02-06 16:02:40,147554,2021-02-06 00:00:00,2021-02-06 16:02:59,2021-02-06 16:03:42,1,1.324600,0.54,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,740.966667
18092,2021-02-06 16:11:36,148323,2021-02-06 00:00:00,2021-02-06 16:11:52,2021-02-06 16:13:21,1,0.029638,0.23,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,749.900000
23756,2021-02-06 17:19:38,153986,2021-02-06 00:00:00,2021-02-06 17:22:20,2021-02-06 17:22:24,1,1.311700,4.69,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,817.933333
32469,2021-02-07 08:03:10,86420,2021-02-07 00:00:00,2021-02-07 08:03:10,2021-02-07 08:04:41,1,0.965400,0.19,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,262.700000
41660,2021-02-07 13:33:05,95611,2021-02-07 00:00:00,2021-02-07 13:33:06,2021-02-07 13:33:15,1,2.511500,0.11,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,574.283333
44538,2021-02-07 13:58:36,98489,2021-02-07 00:00:00,2021-02-07 13:58:38,2021-02-07 13:58:50,1,1.551600,3.87,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,599.800000
46696,2021-02-07 14:19:07,100647,2021-02-07 00:00:00,2021-02-07 14:19:08,2021-02-07 14:19:18,1,0.929400,5.24,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,620.316667
50516,2021-02-07 15:21:35,104468,2021-02-07 00:00:00,2021-02-07 15:22:01,2021-02-07 15:23:02,1,0.322100,3.08,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,682.783333
58446,2021-02-07 16:47:26,112397,2021-02-07 00:00:00,2021-02-07 16:47:27,2021-02-07 16:47:44,1,1.536600,1.15,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,768.633333


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_id              450000 non-null  int64  
 2   order_date            450000 non-null  object 
 3   allot_time            450000 non-null  object 
 4   accept_time           449843 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              450000 non-null  int64  
 8   first_mile_distance   450000 non-null  float64
 9   last_mile_distance    450000 non-null  float64
 10  alloted_orders        433052 non-null  float64
 11  delivered_orders      432659 non-null  float64
 12  cancelled             450000 non-null  int64  
 13  undelivered_orders    432659 non-null  float64
 14  lifetime_order_count  449947 non-null  float64
 15  

In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144844 entries, 0 to 144843
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            144844 non-null  object 
 1   order_id              144844 non-null  int64  
 2   order_date            144844 non-null  object 
 3   allot_time            144844 non-null  object 
 4   accept_time           144776 non-null  object 
 5   rider_id              144844 non-null  int64  
 6   first_mile_distance   144844 non-null  float64
 7   last_mile_distance    144844 non-null  float64
 8   alloted_orders        140071 non-null  float64
 9   delivered_orders      139960 non-null  float64
 10  undelivered_orders    139960 non-null  float64
 11  lifetime_order_count  144066 non-null  float64
 12  reassignment_method   4632 non-null    object 
 13  reassignment_reason   4635 non-null    object 
 14  reassigned_order      4635 non-null    float64
 15  

In [30]:
drop_cols = ['pickup_time', 'delivered_time', 'cancelled_time']
train.drop(drop_cols, axis = 1, inplace = True)

In [31]:
def time_diff(time_list_1, time_list_2):
    hour_diff = (int(time_list_1[0]) - int(time_list_2[0]))*3600
    min_diff = (int(time_list_1[1]) - int(time_list_2[1]))*60
    sec_diff = (int(time_list_1[2]) - int(time_list_2[2]))
    return hour_diff + min_diff + sec_diff
    
def create_time_features(row):
    
    order_times = row['order_time'].split(' ')[1].split(':')
    allot_times = row['allot_time'].split(' ')[1].split(':')
    if row['accept_time'] == row['accept_time']:
        accept_times = row['accept_time'].split(' ')[1].split(':')
    
    row['rider_allot_time'] = time_diff(allot_times, order_times)
    try:
        row['rider_accept_time'] = time_diff(accept_times, allot_times)
    except:
        row['rider_accept_time'] = -1
        
    return row

In [32]:
def apply_func(df):
    df = df.progress_apply(create_time_features, axis = 1)
    return df

def parallelize_dataframe(df, func, n_cores = 10):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [33]:
train = parallelize_dataframe(train, apply_func)
test = parallelize_dataframe(test, apply_func)

100%|████████████████████████████████████████████████████████| 14485/14485 [00:22<00:00, 653.65it/s]


In [34]:
train.head()

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,rider_allot_time,rider_accept_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,24,9
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,41,48
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,8,16
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,32,41
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,51,36


In [35]:
train[train['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,rider_allot_time,rider_accept_time
315699,2021-02-03 06:43:10,239802,2021-02-03 00:00:00,2021-02-03 06:43:25,2021-02-03 06:43:30,1,1.4561,4.14,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,147.533333,15,5
328569,2021-02-03 13:38:38,252675,2021-02-03 00:00:00,2021-02-03 13:38:55,2021-02-03 13:40:17,1,2.4127,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,510.566667,17,82
331400,2021-02-03 14:02:48,255506,2021-02-03 00:00:00,2021-02-03 14:04:44,2021-02-03 14:05:50,1,0.8029,3.68,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,534.733333,116,66
346038,2021-02-03 16:14:20,270142,2021-02-03 00:00:00,2021-02-03 16:14:55,2021-02-03 16:15:50,1,1.1215,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,666.266667,35,55
348876,2021-02-03 16:41:11,272980,2021-02-03 00:00:00,2021-02-03 16:41:47,2021-02-03 16:42:43,1,0.4628,1.51,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,693.116667,36,56
352353,2021-02-03 17:14:48,276457,2021-02-03 00:00:00,2021-02-03 17:15:39,2021-02-03 17:16:26,1,1.1628,1.36,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,726.733333,51,47
353553,2021-02-03 17:26:31,277657,2021-02-03 00:00:00,2021-02-03 17:26:55,2021-02-03 17:27:51,1,1.0028,3.59,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,738.450000,24,56
373828,2021-02-04 14:14:41,216800,2021-02-04 00:00:00,2021-02-04 14:15:41,2021-02-04 14:16:31,1,0.8283,1.91,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,519.200000,60,50
379722,2021-02-04 15:10:50,222695,2021-02-04 00:00:00,2021-02-04 15:11:47,2021-02-04 15:12:24,1,0.2979,0.58,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,575.350000,57,37
383660,2021-02-04 15:48:53,226632,2021-02-04 00:00:00,2021-02-04 15:49:39,2021-02-04 15:49:52,1,0.8439,0.62,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,613.400000,46,13


In [36]:
def train_fix_lifetime_orders(riders):
    master = pd.DataFrame()
    
    prev_rider = ''
    prev_date = ''
    prev_lifetime = 0
    inc = 0
    for i, row in tqdm(riders.iterrows(), total = len(riders)):
        temp  = train[train['rider_id'] == row['rider_id']][train['order_date'] == row['order_date']]
        if row['order_date'] != prev_date:
            inc += prev_lifetime
            prev_lifetime = row['lifetime_order_count']
        if row['rider_id'] != prev_rider:
            inc = 0
        temp['lifetime_order_count'] += inc
        
        prev_date = row['order_date']
        prev_rider = row['rider_id']
        master = master.append(temp)
    return master

In [42]:
def test_fix_lifetime_orders(riders):
    master = pd.DataFrame()
    
    prev_rider = ''
    prev_date = ''
    prev_lifetime = 0
    inc = 0
    for i, row in tqdm(riders.iterrows(), total = len(riders)):
        temp  = test[test['rider_id'] == row['rider_id']][test['order_date'] == row['order_date']]
        if row['order_date'] != prev_date:
            inc += prev_lifetime
            prev_lifetime = row['lifetime_order_count']
        if row['rider_id'] != prev_rider:
            inc = 0
        temp['lifetime_order_count'] += inc
        
        prev_date = row['order_date']
        prev_rider = row['rider_id']
        master = master.append(temp)
    return master

In [41]:
train_riders = train[['rider_id', 'order_date', 'lifetime_order_count']].groupby(['rider_id', 'order_date']).count().reset_index()
train_copy = train_fix_lifetime_orders(train_riders)

100%|███████████████████████████████████████████████████████| 94620/94620 [2:39:50<00:00,  9.87it/s]


In [44]:
test_riders = test[['rider_id', 'order_date', 'lifetime_order_count']].groupby(['rider_id', 'order_date']).count().reset_index()
test_copy = test_fix_lifetime_orders(test_riders)

100%|█████████████████████████████████████████████████████████| 31680/31680 [18:25<00:00, 28.66it/s]


In [45]:
train.to_csv('Data/train_processed.csv', index = False)
test.to_csv('Data/test_processed.csv', index = False)

In [46]:
train = train_copy
test = test_copy

In [49]:
def interp(x):
    d = pd.DataFrame(x['session_time'].values, index = pd.to_datetime(x['order_time']), columns = ['session_time'])
    d = d.interpolate('time')
    d = d.interpolate('bfill')
    
    x['session_time'] = d['session_time'].values
    return x

grp_obj = train.groupby(['order_date', 'rider_id'])
frames = []

for name, grp in tqdm(grp_obj):
    grp = interp(grp)
    frames+=[grp]
    
train = pd.concat(frames)
train.sort_index(inplace = True)

grp_obj = test.groupby(['order_date', 'rider_id'])
frames = []

for name, grp in tqdm(grp_obj):
    grp = interp(grp)
    frames+=[grp]
    
test = pd.concat(frames)
test.sort_index(inplace = True)

train['ratio'] = train['first_mile_distance']/(train['last_mile_distance'] + 1e-7)
test['ratio'] = test['first_mile_distance']/(test['last_mile_distance'] + 1e-7)

sc = MinMaxScaler()
sc = sc.fit(train['ratio'].values.reshape(-1,1))
train['ratio'] = sc.transform(train['ratio'].values.reshape(-1,1))
test['ratio'] = sc.transform(test['ratio'].values.reshape(-1,1))

100%|████████████████████████████████████████████████████████| 31680/31680 [00:54<00:00, 584.55it/s]


In [50]:
more_drop = ['order_time', 'order_date', 'allot_time',
             'accept_time']

train.drop(more_drop, axis = 1, inplace = True)
test.drop(more_drop, axis = 1, inplace = True)

In [51]:
from sklearn.preprocessing import LabelEncoder
text_fill = ['reassignment_method', 'reassignment_reason']
num_fill = ['reassigned_order', 'session_time', 'alloted_orders',
       'delivered_orders', 'undelivered_orders', 'lifetime_order_count',]

for col in text_fill:
    le = LabelEncoder()
    train[col] = train[col].fillna('Missing')
    train[col] = le.fit_transform(train[col])
    test[col] = test[col].fillna('Missing')
    test[col] = le.transform(test[col])

for col in num_fill:
    train[col] = train[col].fillna(-1)
    test[col] = test[col].fillna(-1)

In [52]:
train['reassigned_order'].value_counts()

-1.0    436247
 1.0     13753
Name: reassigned_order, dtype: int64

In [53]:
sc = StandardScaler()
X_data = train.drop(['cancelled', 'order_id'], axis = 1)
y_data = train['cancelled']

kfold = StratifiedKFold(random_state = 10, n_splits = 10, shuffle = True)
splits = kfold.split(X_data, y_data)

sm = SMOTE(random_state = 10, k_neighbors = 5, n_jobs = -1)

roc_scores = []
Models = []

for Train, Test in splits:
    X_Train, X_Val, Y_Train, Y_Val = X_data.iloc[Train], X_data.iloc[Test], y_data.iloc[Train], y_data.iloc[Test]
    xgb = XGBClassifier(n_estimators = 8000, max_depth = 8, n_jobs = -1,
                        learning_rate = 0.05, reg_lambda = 0.1, random_state = 10)
    X_Train, Y_Train = sm.fit_resample(X_Train, Y_Train)
    xgb.fit(X_Train, Y_Train, early_stopping_rounds = 200,
            eval_set = [(X_Train, Y_Train), (X_Val, Y_Val)],
            eval_metric = 'logloss', verbose = True)
    pred = xgb.predict(X_Val)
    
    roc_scores.append(roc_auc_score(pred, Y_Val))
    print("ROC Score: ", np.mean(roc_scores))
    Models.append(xgb)

[0]	validation_0-logloss:0.66377	validation_1-logloss:0.66410
[1]	validation_0-logloss:0.63721	validation_1-logloss:0.63747
[2]	validation_0-logloss:0.61301	validation_1-logloss:0.61299
[3]	validation_0-logloss:0.59092	validation_1-logloss:0.59028
[4]	validation_0-logloss:0.57064	validation_1-logloss:0.56928
[5]	validation_0-logloss:0.55166	validation_1-logloss:0.54940
[6]	validation_0-logloss:0.53431	validation_1-logloss:0.53100
[7]	validation_0-logloss:0.51809	validation_1-logloss:0.51383
[8]	validation_0-logloss:0.50348	validation_1-logloss:0.49805
[9]	validation_0-logloss:0.48957	validation_1-logloss:0.48316
[10]	validation_0-logloss:0.47739	validation_1-logloss:0.46998
[11]	validation_0-logloss:0.46545	validation_1-logloss:0.45681
[12]	validation_0-logloss:0.45511	validation_1-logloss:0.44538
[13]	validation_0-logloss:0.44538	validation_1-logloss:0.43468
[14]	validation_0-logloss:0.43555	validation_1-logloss:0.42368
[15]	validation_0-logloss:0.42761	validation_1-logloss:0.41460
[1

[130]	validation_0-logloss:0.23062	validation_1-logloss:0.20952
[131]	validation_0-logloss:0.22972	validation_1-logloss:0.20885
[132]	validation_0-logloss:0.22945	validation_1-logloss:0.20865
[133]	validation_0-logloss:0.22868	validation_1-logloss:0.20785
[134]	validation_0-logloss:0.22812	validation_1-logloss:0.20746
[135]	validation_0-logloss:0.22726	validation_1-logloss:0.20660
[136]	validation_0-logloss:0.22661	validation_1-logloss:0.20621
[137]	validation_0-logloss:0.22624	validation_1-logloss:0.20596
[138]	validation_0-logloss:0.22595	validation_1-logloss:0.20572
[139]	validation_0-logloss:0.22553	validation_1-logloss:0.20545
[140]	validation_0-logloss:0.22517	validation_1-logloss:0.20520
[141]	validation_0-logloss:0.22424	validation_1-logloss:0.20458
[142]	validation_0-logloss:0.22386	validation_1-logloss:0.20433
[143]	validation_0-logloss:0.22353	validation_1-logloss:0.20411
[144]	validation_0-logloss:0.22283	validation_1-logloss:0.20360
[145]	validation_0-logloss:0.22269	valid

[259]	validation_0-logloss:0.18495	validation_1-logloss:0.17563
[260]	validation_0-logloss:0.18448	validation_1-logloss:0.17528
[261]	validation_0-logloss:0.18423	validation_1-logloss:0.17509
[262]	validation_0-logloss:0.18403	validation_1-logloss:0.17495
[263]	validation_0-logloss:0.18355	validation_1-logloss:0.17462
[264]	validation_0-logloss:0.18329	validation_1-logloss:0.17444
[265]	validation_0-logloss:0.18307	validation_1-logloss:0.17430
[266]	validation_0-logloss:0.18263	validation_1-logloss:0.17401
[267]	validation_0-logloss:0.18232	validation_1-logloss:0.17378
[268]	validation_0-logloss:0.18183	validation_1-logloss:0.17348
[269]	validation_0-logloss:0.18159	validation_1-logloss:0.17333
[270]	validation_0-logloss:0.18150	validation_1-logloss:0.17323
[271]	validation_0-logloss:0.18119	validation_1-logloss:0.17305
[272]	validation_0-logloss:0.18086	validation_1-logloss:0.17282
[273]	validation_0-logloss:0.18075	validation_1-logloss:0.17272
[274]	validation_0-logloss:0.18063	valid

[388]	validation_0-logloss:0.15668	validation_1-logloss:0.15573
[389]	validation_0-logloss:0.15666	validation_1-logloss:0.15570
[390]	validation_0-logloss:0.15649	validation_1-logloss:0.15559
[391]	validation_0-logloss:0.15601	validation_1-logloss:0.15524
[392]	validation_0-logloss:0.15562	validation_1-logloss:0.15499
[393]	validation_0-logloss:0.15535	validation_1-logloss:0.15484
[394]	validation_0-logloss:0.15522	validation_1-logloss:0.15475
[395]	validation_0-logloss:0.15491	validation_1-logloss:0.15456
[396]	validation_0-logloss:0.15469	validation_1-logloss:0.15441
[397]	validation_0-logloss:0.15460	validation_1-logloss:0.15437
[398]	validation_0-logloss:0.15421	validation_1-logloss:0.15411
[399]	validation_0-logloss:0.15399	validation_1-logloss:0.15396
[400]	validation_0-logloss:0.15386	validation_1-logloss:0.15388
[401]	validation_0-logloss:0.15368	validation_1-logloss:0.15376
[402]	validation_0-logloss:0.15359	validation_1-logloss:0.15370
[403]	validation_0-logloss:0.15330	valid

[517]	validation_0-logloss:0.13340	validation_1-logloss:0.14012
[518]	validation_0-logloss:0.13315	validation_1-logloss:0.13991
[519]	validation_0-logloss:0.13306	validation_1-logloss:0.13984
[520]	validation_0-logloss:0.13290	validation_1-logloss:0.13973
[521]	validation_0-logloss:0.13267	validation_1-logloss:0.13960
[522]	validation_0-logloss:0.13251	validation_1-logloss:0.13948
[523]	validation_0-logloss:0.13232	validation_1-logloss:0.13936
[524]	validation_0-logloss:0.13218	validation_1-logloss:0.13926
[525]	validation_0-logloss:0.13214	validation_1-logloss:0.13922
[526]	validation_0-logloss:0.13194	validation_1-logloss:0.13908
[527]	validation_0-logloss:0.13174	validation_1-logloss:0.13896
[528]	validation_0-logloss:0.13167	validation_1-logloss:0.13889
[529]	validation_0-logloss:0.13149	validation_1-logloss:0.13878
[530]	validation_0-logloss:0.13110	validation_1-logloss:0.13848
[531]	validation_0-logloss:0.13104	validation_1-logloss:0.13844
[532]	validation_0-logloss:0.13092	valid

...

[3129]	validation_0-logloss:0.01521	validation_1-logloss:0.06370
[3130]	validation_0-logloss:0.01520	validation_1-logloss:0.06369
[3131]	validation_0-logloss:0.01519	validation_1-logloss:0.06369
[3132]	validation_0-logloss:0.01518	validation_1-logloss:0.06369
[3133]	validation_0-logloss:0.01518	validation_1-logloss:0.06369
[3134]	validation_0-logloss:0.01518	validation_1-logloss:0.06370
[3135]	validation_0-logloss:0.01516	validation_1-logloss:0.06369
[3136]	validation_0-logloss:0.01516	validation_1-logloss:0.06370
[3137]	validation_0-logloss:0.01516	validation_1-logloss:0.06370
[3138]	validation_0-logloss:0.01516	validation_1-logloss:0.06370
[3139]	validation_0-logloss:0.01515	validation_1-logloss:0.06370
[3140]	validation_0-logloss:0.01515	validation_1-logloss:0.06370
[3141]	validation_0-logloss:0.01514	validation_1-logloss:0.06369
[3142]	validation_0-logloss:0.01513	validation_1-logloss:0.06369
[3143]	validation_0-logloss:0.01513	validation_1-logloss:0.06369
[3144]	validation_0-loglo

[3256]	validation_0-logloss:0.01434	validation_1-logloss:0.06368
[3257]	validation_0-logloss:0.01433	validation_1-logloss:0.06368
[3258]	validation_0-logloss:0.01432	validation_1-logloss:0.06367
[3259]	validation_0-logloss:0.01430	validation_1-logloss:0.06366
[3260]	validation_0-logloss:0.01429	validation_1-logloss:0.06366
[3261]	validation_0-logloss:0.01428	validation_1-logloss:0.06365
[3262]	validation_0-logloss:0.01427	validation_1-logloss:0.06364
[3263]	validation_0-logloss:0.01426	validation_1-logloss:0.06364
[3264]	validation_0-logloss:0.01424	validation_1-logloss:0.06365
[3265]	validation_0-logloss:0.01423	validation_1-logloss:0.06363
[3266]	validation_0-logloss:0.01422	validation_1-logloss:0.06364
[3267]	validation_0-logloss:0.01421	validation_1-logloss:0.06364
[3268]	validation_0-logloss:0.01419	validation_1-logloss:0.06365
[3269]	validation_0-logloss:0.01418	validation_1-logloss:0.06364
[3270]	validation_0-logloss:0.01417	validation_1-logloss:0.06364
[3271]	validation_0-loglo

[3383]	validation_0-logloss:0.01346	validation_1-logloss:0.06357
[3384]	validation_0-logloss:0.01346	validation_1-logloss:0.06357
[3385]	validation_0-logloss:0.01346	validation_1-logloss:0.06356
[3386]	validation_0-logloss:0.01345	validation_1-logloss:0.06356
[3387]	validation_0-logloss:0.01345	validation_1-logloss:0.06356
[3388]	validation_0-logloss:0.01344	validation_1-logloss:0.06356
[3389]	validation_0-logloss:0.01342	validation_1-logloss:0.06355
[3390]	validation_0-logloss:0.01342	validation_1-logloss:0.06355
[3391]	validation_0-logloss:0.01342	validation_1-logloss:0.06355
[3392]	validation_0-logloss:0.01341	validation_1-logloss:0.06355
[3393]	validation_0-logloss:0.01339	validation_1-logloss:0.06355
[3394]	validation_0-logloss:0.01338	validation_1-logloss:0.06354
[3395]	validation_0-logloss:0.01337	validation_1-logloss:0.06354
[3396]	validation_0-logloss:0.01337	validation_1-logloss:0.06354
[3397]	validation_0-logloss:0.01337	validation_1-logloss:0.06354
[3398]	validation_0-loglo

[3510]	validation_0-logloss:0.01263	validation_1-logloss:0.06348
[3511]	validation_0-logloss:0.01262	validation_1-logloss:0.06348
[3512]	validation_0-logloss:0.01262	validation_1-logloss:0.06348
[3513]	validation_0-logloss:0.01261	validation_1-logloss:0.06348
[3514]	validation_0-logloss:0.01261	validation_1-logloss:0.06348
[3515]	validation_0-logloss:0.01261	validation_1-logloss:0.06348
[3516]	validation_0-logloss:0.01261	validation_1-logloss:0.06348
[3517]	validation_0-logloss:0.01260	validation_1-logloss:0.06348
[3518]	validation_0-logloss:0.01260	validation_1-logloss:0.06348
[3519]	validation_0-logloss:0.01260	validation_1-logloss:0.06348
[3520]	validation_0-logloss:0.01259	validation_1-logloss:0.06349
[3521]	validation_0-logloss:0.01259	validation_1-logloss:0.06349
[3522]	validation_0-logloss:0.01259	validation_1-logloss:0.06349
[3523]	validation_0-logloss:0.01258	validation_1-logloss:0.06349
[3524]	validation_0-logloss:0.01257	validation_1-logloss:0.06350
[3525]	validation_0-loglo

[3637]	validation_0-logloss:0.01205	validation_1-logloss:0.06352
[3638]	validation_0-logloss:0.01204	validation_1-logloss:0.06352
[3639]	validation_0-logloss:0.01203	validation_1-logloss:0.06353
[3640]	validation_0-logloss:0.01202	validation_1-logloss:0.06353
[3641]	validation_0-logloss:0.01201	validation_1-logloss:0.06352
[3642]	validation_0-logloss:0.01199	validation_1-logloss:0.06352
[3643]	validation_0-logloss:0.01199	validation_1-logloss:0.06352
[3644]	validation_0-logloss:0.01198	validation_1-logloss:0.06351
[3645]	validation_0-logloss:0.01197	validation_1-logloss:0.06351
[3646]	validation_0-logloss:0.01196	validation_1-logloss:0.06351
[3647]	validation_0-logloss:0.01196	validation_1-logloss:0.06352
[3648]	validation_0-logloss:0.01195	validation_1-logloss:0.06351
[3649]	validation_0-logloss:0.01194	validation_1-logloss:0.06351
[3650]	validation_0-logloss:0.01194	validation_1-logloss:0.06351
[3651]	validation_0-logloss:0.01193	validation_1-logloss:0.06351
[3652]	validation_0-loglo

In [54]:
print("ROC-AUC Score: ", np.mean(roc_scores))

ROC-AUC Score:  0.7485704653886838


In [55]:
pred = []
for model in Models:
    pred.append(model.predict_proba((test.drop('order_id', axis = 1))))

print(np.array(pred).shape)
print(np.mean(pred, axis = 0).shape)
pred = np.argmax(np.mean(pred, axis = 0), axis = 1)

(10, 144844, 2)
(144844, 2)


In [56]:
pred.shape

(144844,)

In [57]:
pred_file = {'order_id': test['order_id'], 'cancelled': pred}
pred_file = pd.DataFrame(pred_file)
pred_file.head()

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0


In [58]:
pred_file.to_csv('Submissions/Sub5.csv', index = False)